In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [ ]:
train_data.Age.fillna(train_data.groupby('Pclass')['Age'].transform('mean'),inplace = True)
test_data.Age.fillna(train_data.groupby('Pclass')['Age'].transform('mean'),inplace = True)

In [ ]:
def age_range(row):
    row.Age = 0 if row.Age < 18 else 1 if (18 <= row.Age < 30) else 2 if (30 <= row.Age < 50) else 3 if (50 <= row.Age < 80) else 4
    return row

train_data_copy = train_data.apply(age_range, axis='columns')
test_data_copy = test_data.apply(age_range, axis='columns')

In [ ]:
train_data_copy['Family'] = train_data_copy['SibSp']+train_data['Parch']
train_data_copy.head()

In [ ]:
test_data_copy['Family'] = train_data_copy['SibSp']+train_data['Parch']
test_data_copy.head()

In [ ]:
x = train_data_copy.drop(['PassengerId','Survived','Ticket','Cabin','Name','Fare','SibSp','Parch'],axis =1)
x.head()

In [ ]:
x.isna().sum()

In [ ]:
train_data[train_data.Embarked.isna()]

In [ ]:
x.Embarked.fillna('S',inplace = True)

In [ ]:
X_train = pd.get_dummies(x)
X_train.head(10)
test_data_copy = test_data_copy.drop(['PassengerId','Ticket','Cabin','Name','Fare','SibSp','Parch'],axis =1)
x_test = pd.get_dummies(test_data_copy)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

model = RandomForestClassifier(n_estimators=100,max_depth=5,random_state=3)
model.fit(X_train, y)
predictions = model.predict(x_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)

In [ ]:
# from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

model_tree = tree.DecisionTreeClassifier(max_depth=5, max_leaf_nodes=2)
model_tree = model_tree.fit(X_train, y)
prediction_tree = model_tree.predict(x_test)
output_tree = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': prediction_tree})
output_tree.to_csv('my_submission_tree.csv', index=False)